# [Your title goes here]

# 1 Author

**Student Name:**  Daria Gorbunova

**Student ID**: 231173199



# 2 Problem formulation

## 2.1 Type of problem:
This machine learning problem is a multi-class classification one. The goal is to take a 10-second audio of a hum or a whistle, and determine which of the 8 possible song it is. The possible song labels in this problem are:
1. Happy
2. Try Everything
3. Remember Me
4. New York
5. Friend
6. Necessities
7. Feeling
8. Married

Since the input data (aduio clip) has a label, this is a supervised learning task.

## 2.2 The dataset:
The dataset MLEnd Hums and Whistles II has been provided by the module. The dataset was created with student contribution, where each student was tasked to provide:
- 4 humming instances
- 4 whistling instances

per each of the 8 songs outlined above. 

However, some participants could not whistle and were instead given permission to supply 8 humming instances only. This lead to uneven numbers of hums vs whistles in the raw collected data. 
After the examination of the subset (800 samples) provided for this mini project, using file-counting script:
- The dataset hsa 800 samples
- The number of instances of hum and whistles is balanced equally. 
- Each of the 8 song classes has 100 instances. 

Therefore, the subset of the dataset released by the teaching team for this task is balanced, which reduces the risk of class imbalance, which is ideal for machine learning models

## 2.3 Nature of instances:
Each instance of the dataset consists of human-recorded raw waveform audio of differing lengths. Before providing the dataset, the module team applied basic preprocessubg to clean the raw submission which included trimming of long silences, removing obvious background noise where possible and normalising audio volume.

However, the dataset will still contain substantial real-worls variation due to the nature of the recordings affected by factors such as different microphones, different acousitc environments, just to name a few. 

Audio instances are high dimensional and even 5 second clip smaples at 16kHz contains around 80,000 raw waveform values. For this reason, raw audio cannot be used directly with ML models and feature extraction is required, this will be outlined in methodology. 

## 2.4 What makes the problem interesting:
- High-variability in humming and whistling: people hum and whistle differently, there is varibaility in pitch stability, tempo and rhythm. Even the same person can reproduce the same melody inconsistently. 
- Similarity between some melodies: certain songs in the dataset share similar rhythmic patterns.
- Recording and environemntal noise: audio was captured through the same software, but with different devices, microphones and under different acoustic conditions. Despite preprocessing done by module team, variation remains.
- Complex, high-dimensional input: raw audio is highly dimensional and relationship between raw waveform and melody is higly non-linear, which cannot be learned by classical methods. Featuer extraction is required to represent the instances better.

# 3 Methodology
## 3.1 Overvew of methodology
The overall methodology will follow the standard supervised learning pipeline for audio classification. 
- Loading the dataset. 
- Preprocessing raw audio waveform to extract meaningful features.
- Training a range of machine learning models.
- Optimising hyperparameters using the validation data.
- Evaluating final performance on the test set.
- Comparing models to reflect on results across them. 

## 3.2 Audio preprocessing
As mentioned in the problem formulation, raw waveform data is highly dimensional and not very useful for machine learning algorithms. Preprocessing of the data will include:
- Loading audio files using librosa at a fixed sampling rate of 16khz. 
- Normalising amplitude to reduce loudness variation.
- Converting waveforms into 2D time-frequency representations using MFCCs and Mel spectograms. 
- Trimming to make sure all the instances are 10 seconds long, and have a consistent number of frames. 
- Scalling features.

MFCCs will be used for Logistic Regression, KNN and MLP. 

Mel-spectograms will be used for CNNs to preserve spatial patterns required for convolution. 

## 3.3 Data splitting
To ensure correct evaluation of the models, the larger dataset will be divided into:
- Training set, used to fit model
- Validation set, used for hyperparameter turning and model selection.
- Testing set, used at the end to estimate genralisation performance.

A stratified split will be used to maintian balanced class proportion for all 8 songs. The full methodology and implementation of data splitting is outlined in section 5. 

## 3.4 Models considered
A range of models will be implemented to examine how perfromance changes with model complexity. 
### 3.4.1 Logistic Regression
A simple linear classifier, used for baseline to commpare more advanced models against. Logistic regression is fast to train and interpretable. It directly uses flattened extracted features from data preprocessing. 

Not expected to perform well as humming and whistling data is nonlinear. 
### 3.4.2 k-Nearest Neighbours (KNN)
Non-parametric and memory based model which is easy to implement and useful for examining how the data clusters after feature extraction using MFCC. This model will be sensitive to feature scaling and high-dimensionality but is expected to have better performance depending on k. 
### 3.4.3 Multi-layer Perceptron (MLP)
Fully connected neural network that can model nonlinear relationships in the data. Also works from flattened features extracted using MFCC but has risks of overfitting if hyperparameters are not tuned carefully. 
### 3.4.4 Convolutional Neural Network (CNN)
Deep learning model capable of extracting spatial patterns from 2D representations. highest expected performance due to ability to learn hierachical features. Not as easy to implement due to factors such as regularisation, and more computationally expensive with added risk of overffiting on small datasets. 

## 3.5 Performance Metrics
Model performance will be quanitified using:
- Accuracy to determine the percentage of correctly classigied test samples.
- Confusion matrix to provide insigns into which classes are being misclassified. 
- Loss curves for MLP and CNN to heeelp diagnose overfitting. 

Accuracy, although not always best to evaluate models, is appropriate in this case as this is a balanced multi-class classification task. 

## 3.6 Validation Task
Validation task inloves the selection of the best parameters such as k in KNN and learning rates. Comparing performance across models with these turned paramteres on the validation data. It is also where i will check if the model is overffiting by monitoring training and validation accuracy and selection the model with best performance to take to the testing set.


# 4 Implemented ML prediction pipelines
The main idea of the prediction pipelines in this project is to convert raw aduio recordings from the dataset into inputs suitbal for machine learning models, mapping these representations to one of the eight song labels. 

The pipeline is a sequence of processing stages, each applying some transformation to the data into a new intermediate representation, which finally leads to a class label. Since I am going to implement 4 models, I have two main pipelines:

- Pipeline A: used by Logistic Regression, k-Nearest Neighbours (kNN) and Multi-Layer Perceptron (MLP)
- Pipeline B: used for Convolutional Neural Network

The need for two pipelines comes from the fact that both share the same loading procedure, but differ in feature extraction stages.

## 4.1 Transformation stage
During this stage raw waveform audio data is converted into meaningful features which can be used for machine learning, as original data is too high-dimensional and unsuitable for the task. All audio files are loaded at a fixed sampling rate and their amplitude is normalised, and finally featuers are extracted. 
### 4.1.1 Pipeline A
**Input:** audio waveform instances from the dataset
**Transformations:**
- Audio is trimmed to a fixed 10 second duration.
- Mel-Frequency Cepstral Coefficient (MFCCs) are computed.
- MFCC matrix is flattened by averaging across time interval (10 seconds)
- Features are scaled using StandardScaler
**Output:** Fixed-length feature vector suitable for machine learning models 1-3. 

After the transformation is done, MFCC has compressed the signal into a meaningful space while reducing dimensionality, so data can be used in this task. 
MFCC works by converting the audio into short frames and then looking at which frequencies are resent. The frquency spectrum is then compressed using triangular Mel filterts. Logarithmic values are taken and most important coefficients are kept using DCT.

Since the result is a vector, this makes it ideal for logistic regresson, kNN and MLPs which use vector inputs. 
### 4.1.2 Pipeline B
**Input:** audio waveform instances from the dataset
**Transformations:**
- Audio is trimmed to a fixed 10 second duration.
- Mel-spectogram is generated
- Log scaling is applied to stabilise for variance
- Resulting 2D time-frequency array is normalised
**Output:** 2D array representing a spectogram suitable for a convolution network. 

CNN looks at spatial locality and patterns in the 2D data. Using this method we preserve meaningful time-frequency relationships which are then used by CNN to learns hierarchical audio features which cant be accessed by simpler models. 

## 4.2 Model stage
Eac processed feature representation is passed into a classification model, three of the models use the same input format which are MFCC vectors, and CNN uses a 2D spectogram array.

### 4.2.1 Logistic Regression
**Input:** Flattened MFCC vector

**Output:** Prooability distirbution over 8 classes 

Logistic regression acts as a simple model which can be used as a baseline and benchmarking for other models. It is also used to see if features extracted in previous stage contain enough information to be modeled linearly. 

### 4.2.2 k-Nearest Neighbours (kNN)
**Input:** Flattened MFCC vector

**Output:** Predicted class label from majority vote among k closest samples. 

Reveals natural cluster structure in MFCC (feature) space, non parametric baseline. Makes no assumption about the underlying data line Logistic regression did. 

### 4.2.3 Multi-layer Perceptron (MLP)
**Input:** Flattened MFCC vector

**Architecture:** Dense layers with nonlinear activations, ending with a softmax layer

**Output:** Class probabilities 

Can learn non-linear relationships between MFCC extracted features wihtout requiring convolution. Unlike Logistic Regression, can model more complex relationships as it can learn non linear transformations,potentially expected to perform better. 

### 4.2.4 Convolutional Neural Network (CNN)
**Input:** 2D mel-spectogram array

**Architecture:** Convolution, activation, pooling, followed by dense layers.

**Output:** Class probabilities

Can capture time-frequency patterns and learn hierarcal representations, best for audio classification out of the three previous models.

## 4.3 Ensemble stage

For my mini project I will not impolment an ensemble method. 

Techniques such as baggins, boosting or stacking can improve predicion performance of models as it combines multiple models I chose to train multiple deep learning and classical models instead, as outlined above. 

The aim of my project is to compare individual model families like linear models, instance-based models, neural networks and convolutional networks. Implementing an ensemble stage on top of this would make it harder to evaluate the model families independently. 

Therefore, the model stage insludes four models and pipelines that are described above and the ensemble stage is intentionally left out.

# 5 Dataset

Describe the datasets that you will create to build and evaluate your models. Your datasets need to be based on our MLEnd Hums and Whistles II Dataset. After describing the datasets, build them here. You can explore and visualise the datasets here as well.

If you are building separate training and validatio datasets, do it here. Explain clearly how you are building such datasets, how you are ensuring that they serve their purpose (i.e. they are independent and consist of IID samples) and any limitations you might think of. It is always important to identify any limitations as early as possible. The scope and validity of your conclusions will depend on your ability to understand the limitations of your approach.

If you are exploring different datasets, create different subsections for each dataset and give them a name (e.g. 5.1 Dataset A, 5.2 Dataset B, 5.3 Dataset 5.3) .



# 6 Experiments and results

Carry out your experiments here. Analyse and explain your results. Unexplained results are worthless.

# 7 Conclusions

Your conclusions, suggestions for improvements, etc should go here.

# 8 References

Acknowledge others here (books, papers, repositories, libraries, tools)